# Instructions


### 1. Upload your video file.
### 2. Run the **Lib** command.
### 3. On **App** script:
* On **config (name)** section match the `file_name` variable to the uploaded video's filename. *This var will be used for the final downloadable file (e.g., `my_example_video.mp4`).*
* On **config (quality)** section you can adjust following params:  
    ***Keeping the proposed values balances file quality and size.***
  * `-crf` (Constant Rate Factor → Controls video quality):
    * `0`: Lossless (best quality/huge file).
    * `18–28`: Range for most real-world encodes.
    * `23`: Default.
    * Above `30`: Quality drops off rapidly.
    * Above `51`: Invalid.
  * `-preset` (Encoding speed vs compression → Affects size & efficiency):
    * `ultrafast`: Worst.
    * `fast`: Okay.
    * `medium`: Balanced.
    * `slow`: Better.
    * `veryslow`: Best.
  * `-b:a` (Audio Bitrate → Controls audio quality):
    * Typical values (from worst to best): `128k`, `160k`, `192k`, `256k`, `320k`.
* Run the script.

# Lib

In [ ]:
!pip install tqdm

# App

In [ ]:
import os
import re
import time
import subprocess
from tqdm.notebook import tqdm

# ==== Config (name) ====
input_file = "my_example_video.avi"

# ==== Setup ====
output_file = f"{input_file.split('.', 1)[0]}.mp4"
time_pattern = re.compile(r"time=(\d+:\d+:\d+\.\d+)")
if os.path.exists(output_file):
    os.remove(output_file)

# ==== Config (quality) ====
command = [
    "ffmpeg", "-i", input_file,
    "-c:v", "libx264", "-crf", "10", "-preset", "medium",
    "-c:a", "aac", "-b:a", "320k",
    "-movflags", "+faststart",
    output_file
]

# ==== Helpers ====
def elapsed_time(seconds):
    if not isinstance(seconds, (int, float)) or seconds < 0:
        return "Bad time value"

    if seconds < 60:
        return f"{seconds:.2f} s"
    elif seconds < 3600:
        minutes = seconds / 60
        return f"{minutes:.2f} m"
    else:
        hours = seconds / 3600
        return f"{hours:.2f} h"

def get_duration(input_file):
    command = [
        "ffprobe", "-i", input_file, "-show_entries", "format=duration",
        "-v", "quiet", "-of", "csv=p=0"
    ]

    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    try:
        return float(result.stdout.strip())
    except ValueError:
        return None

# ==== App ====
def converter():
    duration = get_duration(input_file)
    if duration is None:
        print("❌ Unable to get video duration.")
        return False

    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    pbar = tqdm(total=duration, unit="s", desc="Converting", dynamic_ncols=True)

    last_update_time = 0
    for line in process.stderr:
        match = time_pattern.search(line)
        if match:
            h, m, s = map(float, match.group(1).split(":"))
            current_time = h * 3600 + m * 60 + s
            if current_time > last_update_time:
                increment = current_time - last_update_time
                pbar.update(increment)
                last_update_time = current_time

    process.wait()
    pbar.close()

    _, stderr_output = process.communicate()

    if process.returncode != 0:
        print("❌ Conversion failed.")
        print(stderr_output)
        return False
    return True

# ==== Main ====
if __name__ == "__main__":
    start_time = time.time()
    done = converter()
    if done:
        end_time = time.time()
        print(f"Conversion done [{elapsed_time(end_time - start_time)}].")
        print(f"{output_file} saved.")